<a href="https://colab.research.google.com/github/BobSheehan23/Bob_EquiLend_Models/blob/main/Plumbing_Charts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from fredapi import Fred
from alpha_vantage.timeseries import TimeSeries
import os
import time
import zipfile

# Hardcode the API keys
FRED_API_KEY = '7f8e44038ee69c4f78cf71873e85db16'
ALPHA_VANTAGE_API_KEY = 'IOTZFZG01XK55BHI'
EXCEL_FILE_PATH_LIQUIDITY = "/content/rrp_tga_reserves.xlsx"
EXCEL_FILE_PATH_DEALER = "/content/BOGZ1FL663061105Q.xlsx"
BACKUP_ZIP_FILE = "/content/FRED_PLUMBING.zip"

# Initialize the Fred object with the API key
try:
    fred = Fred(api_key=FRED_API_KEY)
    print("FRED API initialized successfully.")
except Exception as e:
    print(f"Error initializing FRED API with hardcoded key: {e}")
    fred = None # Set fred to None if API key is not available

# Initialize Alpha Vantage API
try:
    if ALPHA_VANTAGE_API_KEY:
        # Corrected the typo in the variable name
        ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')
        print("Alpha Vantage API initialized successfully.")
    else:
        print("Warning: ALPHA_VANTAGE_API_KEY is not set. Alpha Vantage data will not be fetched.")
        ts = None
except Exception as e:
    print(f"Error initializing Alpha Vantage API with hardcoded key: {e}")
    ts = None


# --- Global Style Configuration ---
# Use a professional and clean style for all charts
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'DejaVu Sans'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['figure.facecolor'] = '#f8f7f4'
plt.rcParams['axes.facecolor'] = '#ffffff'
plt.rcParams['axes.grid'] = False # Remove grid lines

# Define custom colors
CUSTOM_BLUE = '#006db7'
CUSTOM_ORANGE = '#f57600'


# Corrected FRED Series IDs based on previous attempts and common usage
# Using new IDs provided by the user for Bank Reserves and Dealer Positions
fred_series_ids = {
    'OBFR': 'OBFR',
    'EFFR': 'DFF', # Corrected FRED ID for EFFR
    'SOFR': 'SOFR',
    'ON_RRP_Usage': 'RRPONTSYD',
    'Bank_Reserves': 'TOTRESNS',
    'TGA_Balance': 'WTREGEN',
    'Dealer_Net_Treasury_Positions': 'BOGZ1FL663061105Q',
    'EFFR_Volume': 'EFFRVOL',
    'SOFR_Volume': 'SOFRVOL',
    'OBFR_VOLUME': 'OBFRVOL',
    'SRF_Rate':'DFEDTARU',
    'ON_RRP_Rate':'RRPONTSYAWARD',
}

def fetch_fred_series(series_id, series_name, start_date=None, end_date=None):
    """Fetches a single FRED series and returns a pandas Series."""
    if fred is None:
        print(f"Skipping FRED series {series_name} ({series_id}): FRED API not initialized.")
        return pd.Series(dtype=float)
    try:
        print(f"Attempting to fetch FRED series: {series_name} ({series_id})...")
        data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
        if data is None or data.empty:
            print(f"Warning: No data returned or empty for FRED series ID: {series_id} ({series_name}) from API. Will attempt backup.")
            return pd.Series(dtype=float)
        print(f"Successfully fetched data for {series_name} ({series_id}) from API. Data points: {len(data)}")
        return data
    except Exception as e:
        print(f"Error fetching FRED series {series_id} ({series_name}) from API: {e}. Will attempt backup.")
        return pd.Series(dtype=float)

def load_from_zip_backup(series_id, series_name, zip_file_path, start_date=None, end_date=None):
    """Loads data for a specific FRED series from a CSV within a zip backup."""
    csv_filename = f"{series_id}.csv"
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zf:
            if csv_filename in zf.namelist():
                print(f"Attempting to load data for {series_name} ({series_id}) from backup zip...")
                with zf.open(csv_filename) as f:
                    # Load data, assuming the CSV has a 'DATE' and 'VALUE' column
                    backup_df = pd.read_csv(f)
                    if 'DATE' in backup_df.columns and 'VALUE' in backup_df.columns:
                        backup_df['DATE'] = pd.to_datetime(backup_df['DATE'])
                        backup_df = backup_df.set_index('DATE')['VALUE']
                        if start_date is not None:
                            backup_df = backup_df.loc[backup_df.index >= start_date]
                        if end_date is not None:
                            backup_df = backup_df.loc[backup_df.index <= end_date]
                        print(f"Successfully loaded data for {series_name} ({series_id}) from backup. Data points: {len(backup_df)}")
                        return backup_df
                    else:
                        print(f"Error: CSV {csv_filename} in backup zip does not contain 'DATE' and 'VALUE' columns.")
                        return pd.Series(dtype=float)
            else:
                print(f"Warning: {csv_filename} not found in backup zip for {series_name} ({series_id}).")
                return pd.Series(dtype=float)
    except FileNotFoundError:
        print(f"Error: Backup zip file not found at {zip_file_path}.")
        return pd.Series(dtype=float)
    except Exception as e:
        print(f"Error loading data for {series_name} ({series_id}) from backup zip: {e}")
        return pd.Series(dtype=float)


def fetch_alpha_vantage_series(symbol, series_name, interval='daily', outputsize='full'):
    """Fetches a single Alpha Vantage series and returns a pandas DataFrame."""
    if ts is None:
        print(f"Skipping Alpha Vantage series {series_name} ({symbol}): Alpha Vantage API not initialized.")
        return pd.DataFrame()
    try:
        print(f"Attempting to fetch Alpha Vantage series: {series_name} ({symbol})...")
        # Using get_daily_adjusted for typical stock data, can change based on need
        data, meta_data = ts.get_daily_adjusted(symbol=symbol, outputsize=outputsize)
        if data is None or data.empty:
            print(f"Warning: No data returned or empty for Alpha Vantage symbol: {symbol} ({series_name}) from API.")
            return pd.DataFrame()
        print(f"Successfully fetched data for {series_name} ({symbol}) from API. Data points: {len(data)}")
        # Alpha Vantage returns data in a specific format, need to clean it up
        data.columns = [col.split('. ')[1] for col in data.columns] # Clean up column names
        data.index = pd.to_datetime(data.index) # Ensure index is datetime
        return data
    except Exception as e:
        print(f"Error fetching Alpha Vantage series {symbol} ({series_name}) from API: {e}")
        return pd.DataFrame()


# --- Data Generation Functions (Modified to use FRED and Excel and Alpha Vantage) ---

def generate_rate_data(years=5):
    """Fetches real data for overnight rates and volumes from FRED for a specified number of years, with backup."""
    end_date = pd.Timestamp.now()
    start_date = end_date - pd.DateOffset(years=years)

    rate_data = {}
    for series_name, series_id in {
        'OBFR': 'OBFR',
        'EFFR': 'DFF',
        'SOFR': 'SOFR',
        'EFFR_Volume': 'EFFRVOL',
        'SOFR_Volume': 'SOFRVOL',
        'OBFR_Volume': 'OBFRVOL',
        'SRF_Rate': 'DFEDTARU',
        'ON_RRP_Rate': 'RRPONTSYAWARD',
    }.items():
        data = fetch_fred_series(series_id, series_name, start_date, end_date)
        if data.empty and os.path.exists(BACKUP_ZIP_FILE):
             data = load_from_zip_backup(series_id, series_name, BACKUP_ZIP_FILE, start_date, end_date)
        if not data.empty:
            rate_data[series_name] = data

    df = pd.DataFrame(rate_data)

    if df.empty or not isinstance(df.index, pd.DatetimeIndex):
        print("Warning: No or invalid rate data fetched from FRED or backup. Returning empty DataFrame.")
        return pd.DataFrame(columns=['OBFR', 'EFFR', 'SOFR', 'EFFR_Volume', 'SOFR_Volume', 'OBFR_Volume', 'ON_RRP_Rate', 'SRF_Rate'])

    df = df.ffill().bfill()

    # Add policy rates (prioritize fetched data if available, otherwise use hardcoded)
    # Check if the fetched columns exist before using combine_first
    if 'ON_RRP_Rate' not in df.columns or df['ON_RRP_Rate'].dropna().empty:
         df['ON_RRP_Rate'] = pd.Series(5.25, index=df.index) # Use hardcoded if fetch failed
    if 'SRF_Rate' not in df.columns or df['SRF_Rate'].dropna().empty:
         df['SRF_Rate'] = pd.Series(5.50, index=df.index) # Use hardcoded if fetch failed


    # Ensure the DataFrame covers the specified years
    # Interpolate daily data to handle missing days before resampling
    df = df.resample('D').interpolate(method='time').asfreq('B').ffill()
    return df.loc[start_date:end_date].dropna(how='all')


def fetch_fred_liquidity_data(years=5):
    """Fetches liquidity data (Bank Reserves, TGA, ON RRP) from FRED."""
    end_date = pd.Timestamp.now().normalize()
    start_date = end_date - pd.DateOffset(years=years)

    liquidity_data = {}
    # Use the FRED IDs already defined
    series_ids_to_fetch = {
        'ON_RRP_Usage': 'RRPONTSYD',
        'Bank_Reserves': 'TOTRESNS',
        'TGA_Balance': 'WTREGEN',
    }

    for series_name, series_id in series_ids_to_fetch.items():
        data = fetch_fred_series(series_id, series_name, start_date, end_date)
        if data.empty and os.path.exists(BACKUP_ZIP_FILE):
             data = load_from_zip_backup(series_id, series_name, BACKUP_ZIP_FILE, start_date, end_date)
        if not data.empty:
            liquidity_data[series_name] = data

    df = pd.DataFrame(liquidity_data)

    if df.empty or not isinstance(df.index, pd.DatetimeIndex):
        print("Warning: No or invalid liquidity data fetched from FRED or backup. Returning empty DataFrame.")
        return pd.DataFrame(columns=['ON_RRP_Usage', 'Bank_Reserves', 'TGA_Balance'])

    # Convert units to Trillions (based on FRED descriptions - typically Billions)
    for col in ['ON_RRP_Usage', 'Bank_Reserves', 'TGA_Balance']:
        if col in df.columns and not df[col].dropna().empty:
            df[col] = df[col] / 1000.0 # Convert Billions to Trillions

    # Handle missing values (forward fill, then back fill)
    df = df.ffill().bfill()

    # Resample to monthly and take the last value of the month, then filter to last 'years'
    # FRED data for these series is typically weekly or daily, resampling to monthly for Chart 3/4
    df = df.resample('ME').last().dropna(how='all')
    return df.loc[start_date:end_date].dropna(how='all')


def fetch_fred_dealer_position_data(years=5):
    """Fetches dealer position data from FRED."""
    end_date = pd.Timestamp.now().normalize()
    start_date = end_date - pd.DateOffset(years=years)

    dealer_data = {}
    # Use the FRED ID already defined
    series_id_to_fetch = 'BOGZ1FL663061105Q'
    series_name = 'Net_Treasury_Positions'

    data = fetch_fred_series(series_id_to_fetch, series_name, start_date, end_date)
    if data.empty and os.path.exists(BACKUP_ZIP_FILE):
         data = load_from_zip_backup(series_id_to_fetch, series_name, BACKUP_ZIP_FILE, start_date, end_date)

    if not data.empty:
        dealer_data[series_name] = data

    df = pd.DataFrame(dealer_data)

    if df.empty or not isinstance(df.index, pd.DatetimeIndex):
        print("Warning: No or invalid dealer position data fetched from FRED or backup. Returning empty DataFrame.")
        return pd.DataFrame(columns=['Net_Treasury_Positions'])

    # Data from this FRED series is likely in Billions, no unit conversion needed for the plot

    # Handle missing values (forward fill, then back fill)
    df = df.ffill().bfill()

    # Resample to the appropriate frequency (likely quarterly based on FRED series) and filter to last 'years'
    # Assuming quarterly data frequency based on the original FRED series ID format
    df = df.resample('QS-JAN').last().dropna(how='all') # Resample to Quarterly Start (Jan) and take last value
    return df.loc[start_date:end_date].dropna(how='all')


def generate_synthesis_data(years=5):
    """Generates illustrative data for the synthesis chart (no FRED series) for a specified number of years."""
    end_date = pd.Timestamp.now().normalize()
    start_date = end_date - pd.DateOffset(years=years)
    dates = pd.to_datetime(pd.date_range(start=start_date, end=end_date, freq='B'))

    # Create a PLI that shows some stress events
    base_pli = 20 + np.linspace(0, 30, len(dates))
    stress_event1 = np.exp(-((np.arange(len(dates)) - len(dates)*0.3)**2) / (2 * (len(dates)*0.05)**2)) * 40
    stress_event2 = np.exp(-((np.arange(len(dates)) - len(dates)*0.7)**2) / (2 * (len(dates)*0.08)**2)) * 50
    pli = base_pli + stress_event1 + stress_event2 + np.random.normal(0, 2, size=len(dates))

    df = pd.DataFrame({'PLI': pli}, index=dates)

    # Create a SecFin index that reacts negatively to PLI with a lag
    df['SecFin_Performance'] = 100 - (df['PLI'].rolling(window=5).mean().shift(3) * 0.5) + np.random.normal(0, 1, size=len(dates))
    df['SecFin_Performance'] = df['SecFin_Performance'].fillna(100)

    return df

def generate_alpha_vantage_data(years=5):
    """Fetches relevant data from Alpha Vantage for a specified number of years."""
    # Define relevant Alpha Vantage symbols
    # Example: SPY (SPDR S&P 500 ETF Trust) as a proxy for market sentiment
    # You can add more symbols here based on what's helpful for your analysis
    alpha_vantage_symbols = {
        'SPY': 'SPDR S&P 500 ETF Trust',
        # Add more symbols here, e.g., for volatility (VIX if available via API or a proxy)
    }

    all_alpha_vantage_data = {}
    for symbol, name in alpha_vantage_symbols.items():
        # Introduce a small delay between API calls to avoid hitting rate limits
        time.sleep(1) # 1 second delay
        data = fetch_alpha_vantage_series(symbol, name, outputsize='full')
        if not data.empty:
            all_alpha_vantage_data[symbol] = data

    # Combine all fetched Alpha Vantage data into a single DataFrame
    # Check if all_alpha_vantage_data is empty before concatenating
    if not all_alpha_vantage_data:
        print("No Alpha Vantage data fetched or available.")
        return pd.DataFrame()

    combined_alpha_vantage_df = pd.concat(all_alpha_vantage_data.values(), axis=1, keys=all_alpha_vantage_data.keys())

    if combined_alpha_vantage_df.empty:
        print("Combined Alpha Vantage data is empty.")
        return pd.DataFrame()

    # Filter data to the last 'years'
    end_date = pd.Timestamp.now().normalize()
    start_date = end_date - pd.DateOffset(years=years)
    combined_alpha_vantage_df = combined_alpha_vantage_df.loc[start_date:end_date]

    # You may want to select specific columns from the Alpha Vantage data, e.g., 'close'
    # For example, to get only the closing price for SPY:
    if 'SPY' in combined_alpha_vantage_df.columns and 'close' in combined_alpha_vantage_df['SPY'].columns:
         combined_alpha_vantage_df['SPY_Close'] = combined_alpha_vantage_df[('SPY', 'close')]
         # Drop the multi-level columns if you've extracted what you need
         combined_alpha_vantage_df = combined_alpha_vantage_df[['SPY_Close']]


    # Handle missing values (forward fill, then back fill)
    combined_alpha_vantage_df = combined_alpha_vantage_df.ffill().bfill()


    return combined_alpha_vantage_df.dropna(how='all')


# --- Charting Functions ---

def plot_chart_1_rate_corridor(data):
    """Plots Chart 1: Overnight Rates vs. The Fed's Corridor using combined data."""
    fig, ax1 = plt.subplots(figsize=(12, 7))

    # Primary Y-axis for Rates
    color1 = CUSTOM_BLUE # Using custom blue for rates
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Rate (%)', color=color1)
    if 'OBFR' in data.columns and not data['OBFR'].dropna().empty:
         ax1.plot(data.index, data['OBFR'], label='OBFR (Rate)', color='#f97316', lw=2) # Keeping original colors for now
    if 'EFFR' in data.columns and not data['EFFR'].dropna().empty:
         ax1.plot(data.index, data['EFFR'], label='EFFR (Rate)', color='#22c55e', lw=2) # Keeping original colors for now
    if 'SOFR' in data.columns and not data['SOFR'].dropna().empty:
         ax1.plot(data.index, data['SOFR'], label='SOFR (Rate)', color=CUSTOM_BLUE, lw=2) # Using custom blue

    # Add policy rates
    # Use data from the DataFrame if available, otherwise use default hardcoded values
    on_rrp_rate = data['ON_RRP_Rate'].dropna().iloc[-1] if 'ON_RRP_Rate' in data.columns and not data['ON_RRP_Rate'].dropna().empty else 5.25
    srf_rate = data['SRF_Rate'].dropna().iloc[-1] if 'SRF_Rate' in data.columns and not data['SRF_Rate'].dropna().empty else 5.50

    ax1.axhline(srf_rate, color='#ef4444', ls='--', lw=2, label=f'SRF Rate ({srf_rate:.2f}%)')
    ax1.axhline(on_rrp_rate, color='#0ea5e9', ls='--', lw=2, label=f'ON RRP Rate ({on_rrp_rate:.2f}%)')

    # Fill between policy rates
    # Ensure the fill area is aligned with the data index
    fill_data = data[['ON_RRP_Rate', 'SRF_Rate']].dropna(how='all')
    if not fill_data.empty:
        ax1.fill_between(fill_data.index, fill_data['ON_RRP_Rate'], fill_data['SRF_Rate'], color='gray', alpha=0.1, label='Policy Corridor')
    elif not data.empty:
        # If no data for policy rates, fill between the hardcoded values across the plot's x-range using the data index
        ax1.fill_between(data.index, on_rrp_rate, srf_rate, color='gray', alpha=0.1, label='Policy Corridor')


    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax1.grid(False) # Ensure no grid lines

    # No secondary axis for volume on this chart based on user feedback

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    ax1.legend(lines1, labels1, loc='best')

    ax1.set_title('Chart 1: Overnight Rates vs. The Fed\'s Policy Corridor')
    fig.tight_layout()
    return fig

def plot_chart_2_funding_spreads(data):
    """Plots Chart 2: Key Funding Spreads using combined data (Volume removed)."""
    fig, ax1 = plt.subplots(figsize=(12, 7))

    # Primary Y-axis for Spreads
    color1 = CUSTOM_ORANGE
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Spread (Basis Points)', color=color1)

    # Calculate spreads using columns from the combined data
    if 'OBFR' in data.columns and 'SOFR' in data.columns and not data[['OBFR', 'SOFR']].dropna().empty:
        obfr_sofr_spread = (data['OBFR'] - data['SOFR']) * 100
        ax1.plot(data.index, obfr_sofr_spread, label='OBFR - SOFR Spread', color=CUSTOM_ORANGE, lw=2)

    # Using ON RRP Rate from the DataFrame for spread calculation
    if 'SOFR' in data.columns and 'ON_RRP_Rate' in data.columns and not data[['SOFR', 'ON_RRP_Rate']].dropna().empty:
         # Ensure both series are aligned by index before calculating the spread
         temp_spread_df = data[['SOFR', 'ON_RRP_Rate']].dropna()
         sofr_onrrp_spread = (temp_spread_df['SOFR'] - temp_spread_df['ON_RRP_Rate']) * 100
         # Use the latest ON RRP Rate for the label
         latest_on_rrp_rate = data['ON_RRP_Rate'].dropna().iloc[-1] if not data['ON_RRP_Rate'].dropna().empty else 5.25
         ax1.plot(sofr_onrrp_spread.index, sofr_onrrp_spread, label=f'SOFR - ON RRP Rate ({latest_on_rrp_rate:.2f}%) Spread', color=CUSTOM_BLUE, lw=2)


    ax1.axhline(0, color='black', lw=0.5, ls='--')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax1.grid(False) # Ensure no grid lines

    # Remove secondary Y-axis and volume plots
    # ax2 = ax1.twinx()
    # color2 = '#14b8a6' # Using a different color for volume
    # ax2.set_ylabel('Volume (Billions of USD)', color=color2)

    # if 'EFFR_Volume' in data.columns and not data['EFFR_Volume'].dropna().empty:
    #     ax2.plot(data.index, data['EFFR_Volume'], label='EFFR Volume', color='#9b5cf6', ls=':', lw=2) # Using a different color/style for volume
    # if 'SOFR_Volume' in data.columns and not data['SOFR_Volume'].dropna().empty:
    #     ax2.plot(data.index, data['SOFR_Volume'], label='SOFR Volume', color='#3498db', ls=':', lw=2) # Using a different color/style for volume

    # ax2.tick_params(axis='y', labelcolor=color2)
    # ax2.grid(False) # Ensure no grid lines

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    # lines2, labels2 = ax2.get_legend_handles_labels() # Removed
    ax1.legend(lines1, labels1, loc='best') # Updated legend call


    ax1.set_title('Chart 2: Key Funding Spreads') # Updated title
    fig.tight_layout()
    return fig

def plot_chart_3_fed_liabilities(data):
    """Plots Chart 3: Anatomy of the Fed's Key Liabilities using combined data."""
    fig, ax = plt.subplots(figsize=(12, 7))

    labels = []
    series_to_plot = []
    colors = []

    # Columns are expected to be converted to Trillions before being passed to the plotting function
    if 'Bank_Reserves' in data.columns and not data['Bank_Reserves'].dropna().empty:
        labels.append("Bank Reserves")
        series_to_plot.append(data['Bank_Reserves'])
        colors.append(CUSTOM_BLUE)
    if 'ON_RRP_Usage' in data.columns and not data['ON_RRP_Usage'].dropna().empty:
        labels.append("ON RRP Usage")
        series_to_plot.append(data['ON_RRP_Usage'])
        colors.append('#14b8a6')
    if 'TGA_Balance' in data.columns and not data['TGA_Balance'].dropna().empty:
        labels.append("TGA Balance")
        series_to_plot.append(data['TGA_Balance'])
        colors.append(CUSTOM_ORANGE)

    if series_to_plot:
        # Align indices before stacking - important if there are NaNs or different frequencies
        # Use outer join to ensure all dates are included
        combined_series_df = pd.concat(series_to_plot, axis=1)
        combined_series_df = combined_series_df.dropna(how='all') # Drop rows where all are NaN

        if not combined_series_df.empty:
             ax.stackplot(combined_series_df.index, *[combined_series_df[col] for col in combined_series_df.columns], labels=labels, colors=colors)
             ax.set_title('Chart 3: Anatomy of the Fed\'s Key Liabilities')
             ax.set_ylabel('Trillions of USD')
             ax.legend(loc='upper left')
             ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
             ax.grid(False)
             fig.tight_layout()
        else:
            fig.text(0.5, 0.5, 'No data available for Chart 3', horizontalalignment='center', verticalalignment='center', fontsize=16)
            ax.set_title('Chart 3: Anatomy of the Fed\'s Key Liabilities (No Data)')
            ax.axis('off')

    else:
        fig.text(0.5, 0.5, 'No data available for Chart 3', horizontalalignment='center', verticalalignment='center', fontsize=16)
        ax.set_title('Chart 3: Anatomy of the Fed\'s Key Liabilities (No Data)')
        ax.axis('off')

    return fig


def plot_chart_4_mmf_allocation(data):
    """Plots Chart 4: MMF Cash Allocation (ON RRP Usage) using combined data."""
    fig, ax = plt.subplots(figsize=(12, 7))

    # Using the ON RRP Usage from the combined data, which is expected to be in Trillions from the FRED fetch,
    # but converting back to Billions for this specific chart's Y-axis label as requested previously.
    if 'ON_RRP_Usage' in data.columns and not data['ON_RRP_Usage'].dropna().empty:
        # Convert Trillions back to Billions for this specific chart's Y-axis label
        on_rrp_billions = data['ON_RRP_Usage'].dropna() * 1000

        ax.bar(on_rrp_billions.index, on_rrp_billions, width=20, color='#14b8a6', label='ON RRP Facility Usage')
        ax.set_title('Chart 4: MMF Cash Allocation (ON RRP Usage)')
        ax.set_ylabel('Billions of USD')
        ax.legend(loc='upper left')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.grid(False)
        fig.tight_layout()
    else:
        fig.text(0.5, 0.5, 'No data available for Chart 4', horizontalalignment='center', verticalalignment='center', fontsize=16)
        ax.set_title('Chart 4: MMF Cash Allocation (ON RRP Usage) (No Data)')
        ax.axis('off')

    return fig


def plot_chart_5_dealer_inventory(data):
    """Plots Chart 5: Primary Dealer Net Treasury Positions using combined data (Real from FRED)."""
    fig, ax = plt.subplots(figsize=(12, 7))

    # This chart now uses real data from the combined DataFrame fetched via FRED series BOGZ1FL663061105Q
    if 'Net_Treasury_Positions' in data.columns and not data['Net_Treasury_Positions'].dropna().empty:
        dealer_positions = data['Net_Treasury_Positions'].dropna()
        ax.plot(dealer_positions.index, dealer_positions, color='#8b5cf6', lw=2)
        ax.set_title('Chart 5: Primary Dealer Net Treasury Positions')
        # Data from FRED series BOGZ1FL663061105Q is in Billions
        ax.set_ylabel('Billions of USD')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.grid(False)
        fig.tight_layout()
    else:
        fig.text(0.5, 0.5, 'No data available for Chart 5', horizontalalignment='center', verticalalignment='center', fontsize=16)
        ax.set_title('Chart 5: Primary Dealer Net Treasury Positions (No Data)')
        ax.axis('off')

    return fig


def plot_chart_6_stir_futures(data):
    """Plots Chart 6: Market Expectations for Future Policy (Illustrative)."""
    fig, ax = plt.subplots(figsize=(12, 7))

    # This chart will remain illustrative for now unless a suitable API is found
    maturities = ['+3M', '+6M', '+9M', '+12M', '+18M', '+24M']
    implied_rates = [5.25, 5.15, 5.0, 4.8, 4.5, 4.4]
    # Use the latest EFFR or SOFR as a proxy for current rate if available, otherwise use a hardcoded value
    current_rate = data['EFFR'].dropna().iloc[-1] if 'EFFR' in data.columns and not data['EFFR'].dropna().empty else (data['SOFR'].dropna().iloc[-1] if 'SOFR' in data.columns and not data['SOFR'].dropna().empty else 5.33)


    ax.plot(maturities, implied_rates, marker='o', ls='-', color='#eab308', lw=2, label='STIR Futures Implied Rate')
    ax.axhline(current_rate, color='gray', ls='--', lw=2, label=f'Current Policy Rate ({current_rate:.2f}%)')

    ax.set_title('Chart 6: Market Expectations for Future Policy (Illustrative)')
    ax.set_ylabel('Implied Rate (%)')
    ax.set_xlabel('Future Maturity')
    ax.legend()
    ax.grid(False)
    fig.tight_layout()
    return fig

def plot_chart_7_pli(data):
    """Plots Chart 7: The Plumbing Liquidity Index (PLI) (Illustrative) using combined data."""
    fig, ax = plt.subplots(figsize=(12, 7))

    # This chart uses illustrative data from the combined DataFrame
    if 'PLI' in data.columns and not data['PLI'].dropna().empty:
        pli_data = data['PLI'].dropna()
        ax.plot(pli_data.index, pli_data, color=CUSTOM_ORANGE, lw=2, label='PLI')

        # Use the index of the actual data for fill_between
        ax.fill_between(pli_data.index, 0, 40, color='green', alpha=0.1, label='Calm')
        ax.fill_between(pli_data.index, 40, 70, color='yellow', alpha=0.1, label='Stressed')
        ax.fill_between(pli_data.index, 70, 100, color='red', alpha=0.1, label='Crisis')

        ax.set_title('Chart 7: The Plumbing Liquidity Index (PLI) (Illustrative)')
        ax.set_ylabel('Index Level (0-100)')
        ax.set_ylim(0, 100)
        ax.legend(loc='upper left')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
        ax.grid(False)
        fig.tight_layout()
    else:
        fig.text(0.5, 0.5, 'No data available for Chart 7', horizontalalignment='center', verticalalignment='center', fontsize=16)
        ax.set_title('Chart 7: The Plumbing Liquidity Index (PLI) (Illustrative) (No Data)')
        ax.axis('off')

    return fig


def plot_chart_8_synthesis(data):
    """Plots Chart 8: Plumbing Stress vs. Securities Finance Performance (Illustrative) using combined data."""
    fig, ax1 = plt.subplots(figsize=(12, 7))

    # This chart uses illustrative data from the combined DataFrame
    if 'PLI' in data.columns and 'SecFin_Performance' in data.columns and not data[['PLI', 'SecFin_Performance']].dropna().empty:

        # Align indices before plotting
        temp_df = data[['PLI', 'SecFin_Performance']].dropna()

        if not temp_df.empty:
            color1 = CUSTOM_ORANGE
            ax1.set_xlabel('Date')
            ax1.set_ylabel('PLI (Stress)', color=color1)
            ax1.plot(temp_df.index, temp_df['PLI'], color=color1, lw=2, label='PLI')
            ax1.tick_params(axis='y', labelcolor=color1)
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

            ax2 = ax1.twinx()
            color2 = CUSTOM_BLUE
            ax2.set_ylabel('SecFin Performance Index', color=color2)
            ax2.plot(temp_df.index, temp_df['SecFin_Performance'], color=color2, lw=2, label='SecFin Performance')
            ax2.tick_params(axis='y', labelcolor=color2)

            fig.suptitle('Chart 8: Plumbing Stress vs. Securities Finance Performance (Illustrative)')
            ax1.grid(False)
            ax2.grid(False)
            fig.tight_layout(rect=[0, 0, 1, 0.96])
        else:
            fig.text(0.5, 0.5, 'No data available for Chart 8', horizontalalignment='center', verticalalignment='center', fontsize=16)
            fig.suptitle('Chart 8: Plumbing Stress vs. Securities Finance Performance (Illustrative) (No Data)')
            ax1.axis('off')
            # If twinx created a second axis, hide it too - check if ax2 exists and is different from ax1
            if 'ax2' in locals() and ax1 != ax2:
                 ax2.axis('off')

    else:
        fig.text(0.5, 0.5, 'No data available for Chart 8', horizontalalignment='center', verticalalignment='center', fontsize=16)
        fig.suptitle('Chart 8: Plumbing Stress vs. Securities Finance Performance (Illustrative) (No Data)')
        ax1.axis('off')
        # If twinx created a second axis, hide it too - check if ax2 exists and is different from ax1
        if 'ax2' in locals() and ax1 != ax2:
             ax2.axis('off')


    return fig

def plot_chart_9_volumes(data):
    """Plots Chart 9: Overnight Rate Volumes using combined data."""
    fig, ax1 = plt.subplots(figsize=(12, 7))

    # Primary Y-axis for SOFR Volume
    color1 = CUSTOM_BLUE
    ax1.set_xlabel('Date')
    ax1.set_ylabel('SOFR Volume (Billions of USD)', color=color1)
    if 'SOFR_Volume' in data.columns and not data['SOFR_Volume'].dropna().empty:
        ax1.plot(data.index, data['SOFR_Volume'], label='SOFR Volume', color=color1, lw=2)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax1.grid(False) # Ensure no grid lines


    # Secondary Y-axis for EFFR and OBFR Volumes
    ax2 = ax1.twinx()
    color2 = CUSTOM_ORANGE
    ax2.set_ylabel('EFFR and OBFR Volume (Billions of USD)', color=color2)

    if 'EFFR_Volume' in data.columns and not data['EFFR_Volume'].dropna().empty:
        ax2.plot(data.index, data['EFFR_Volume'], label='EFFR Volume', color='#9b5cf6', ls=':', lw=2) # Using a different color/style for volume
    if 'OBFR_Volume' in data.columns and not data['OBFR_Volume'].dropna().empty:
        ax2.plot(data.index, data['OBFR_Volume'], label='OBFR Volume', color='#f97316', ls=':', lw=2) # Example color for OBFR Volume


    ax2.tick_params(axis='y', labelcolor=color2)
    ax2.grid(False) # Ensure no grid lines

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='best')


    ax1.set_title('Chart 9: Overnight Rate Volumes')
    fig.tight_layout()
    return fig


if __name__ == '__main__':
    # Generate necessary data
    rate_data = generate_rate_data(years=5)
    # Fetch liquidity data from FRED
    liquidity_data = fetch_fred_liquidity_data(years=5)
    # Fetch dealer data from FRED
    dealer_data = fetch_fred_dealer_position_data(years=5)
    synthesis_data = generate_synthesis_data(years=5) # Illustrative
    alpha_vantage_data = generate_alpha_vantage_data(years=5) # Fetch Alpha Vantage data


    # Print head and info of dataframes for inspection
    print("\n--- Inspection of rate_data DataFrame (for Charts 1, 2, 9) ---")
    display(rate_data.head())
    rate_data.info()

    print("\n--- Inspection of liquidity_data DataFrame (for Charts 3 and 4) ---")
    display(liquidity_data.head())
    liquidity_data.info()

    print("\n--- Inspection of dealer_data DataFrame (for Chart 5) ---")
    display(dealer_data.head())
    dealer_data.info()

    print("\n--- Inspection of alpha_vantage_data DataFrame ---")
    display(alpha_vantage_data.head())
    alpha_vantage_data.info()


    # Consolidate all data into a single DataFrame
    combined_df = rate_data.join(liquidity_data, how='outer').join(dealer_data, how='outer').join(synthesis_data, how='outer').join(alpha_vantage_data, how='outer')

    # Print info of combined_df to see all columns
    print("\n--- Inspection of combined_df DataFrame ---")
    combined_df.info()


    # Create and display all charts
    fig1 = plot_chart_1_rate_corridor(combined_df)
    fig2 = plot_chart_2_funding_spreads(combined_df) # Includes volume on dual axis
    fig3 = plot_chart_3_fed_liabilities(combined_df) # Uses data from FRED
    fig4 = plot_chart_4_mmf_allocation(combined_df) # Uses data from FRED (ON_RRP_Usage)
    fig5 = plot_chart_5_dealer_inventory(combined_df) # Uses real FRED data
    fig6 = plot_chart_6_stir_futures(combined_df) # Illustrative (updated to use latest rate from data)
    fig7 = plot_chart_7_pli(combined_df) # Illustrative
    fig8 = plot_chart_8_synthesis(combined_df) # Illustrative
    fig9 = plot_chart_9_volumes(combined_df) # New chart for volumes

    # Show all plots
    plt.show()

In [ ]:
!pip install fredapi alpha_vantage